In [172]:
import pandas as pd

In [173]:
df = pd.read_csv("/Users/gaoangw/Downloads/full_data2.csv")

In [174]:
top1pair_perYear = pd.read_csv("/Users/gaoangw/Downloads/top1pair_perYear.csv")

In [175]:
top15 = pd.read_csv("/Users/gaoangw/Downloads/top15.csv")

In [176]:
top15.columns = ['name','Spouse Name', 'Count_overall']

In [177]:
top15

,name,Spouse Name,Count_overall
0,Aaron,Jennifer,110
1,Aaron,Sarah,61
2,Aaron,Amy,56
3,Aaron,Jessica,48
4,Aaron,Katherine,43
...,...,...,...
13564,Zara,Sergey,6
13565,Zhanna,Aleksandr,6
13566,Zoe,Thomas,7
13567,Zoe,Matthew,6


In [179]:
df.columns = ['name','Spouse Name', 'Marriage Year', 'Count','Percentage']

In [180]:
df = df.merge(top15)

In [129]:
df = df.sort_values (by = ['name','Count_overall','Marriage Year'], ascending=[False, False, True])

In [182]:
df.to_csv('df.csv', index = False)

In [138]:
fig = px.line(df[df.name== 'John'], x="Marriage Year", y="Count", color="Spouse Name")
fig.show()

In [5]:
from jupyter_dash import JupyterDash

In [6]:
import dash
from dash import dcc
from dash import html

In [8]:
JupyterDash.infer_jupyter_proxy_config()

In [9]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [98]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server



app.layout = html.Div([
   
    html.H1(children='Couple Names Voyager'),

    html.Div(children='''
        Enter a first name, and discover the evolving pattern of their spouse's name over time! 
    '''),
    
    html.Div([

        html.Div([
            dcc.Dropdown(
                df['name'].unique(),
                'John',
                id='input_name', 
                placeholder="Type/Choose a first name"
            )
        ], style={'width': '48%', 'display': 'inline-block'}),
    ]),

    dcc.Graph(id='pie'),
    dcc.Graph(id='graphic')
])


@app.callback(
    Output('pie', 'figure'),
    Input('input_name', 'value'))
def update_graph(column_name):
    dff = top15[top15['name'] == column_name]
    fig = px.pie(dff, values='Count', names='Spouse Name', title= column_name + "'s Top Spouse Names")
    fig.update_traces(textposition='inside', textinfo='percent+label', showlegend=False)
    return fig


@app.callback(
    Output('graphic', 'figure'),
    Input('input_name', 'value'))
def update_graph2(column_name):
    import plotly.graph_objects as go
    fig = go.Figure()
    dff = top15[top15['name'] == column_name]
    for nameb in dff["Spouse Name"].tolist():
        dff2 = df[(df["Spouse Name"] == nameb) & (df['name'] == column_name)]
        fig.add_trace(go.Scatter(x=dff2["Marriage Year"], y=dff2["Count"],
                        mode='lines',
                        name=nameb))
    fig.update_layout(title= column_name + "'s spouse name pattern over time",
                     xaxis_title="Marriage Year",
                    yaxis_title="Count", 
                     legend_title="Top Spouse Names")
    return fig



In [170]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server


markdown_text1 = '''
- We showcase a 163-year trend of marriage couple names in King County, providing a fun and informative way to explore historical naming trends for couples.
- With this tool, you can delve into the fascinating world of marriage names and discover how they've changed over time.
- To reveal the underlying name patterns and uncover people's (subconscious) name preferences.
- A deeper understanding of the cultural, historical, and social aspects that influence name choices in relationships.
'''

markdown_text2 = '''
### Overall Trend 
- By showing the top 1 marriage couple name per year, you can discover how they've changed over time. Additionally,
you can check [Baby Names Voyager](https://namerology.com/baby-name-grapher/) to see how those popular names become top marriage names 30 years later.
'''


markdown_text3 = '''
### Cultural and Historical Trend
- Now let's play around the data (1.2 million marriage records) and see the cultural, historical, and social aspects that influence name choices in relationships.
- You can enter a Latin American/Asian/Arabic name and see their changes over time because of racial population makeup shift
- You can enter your own first name to see if it make sense to you! 
'''


app.layout = html.Div([
   
    html.H1(children='Couple Names Voyager'),
    html.Div([
    dcc.Markdown(children=markdown_text1)
]),
        html.Div([
    dcc.Markdown(children=markdown_text2)
]),
    dcc.Graph(id='overall'),
    
    html.Div([
    dcc.Markdown(children=markdown_text3)
]),

    html.Div(children='''
        Enter a first name, and discover the evolving pattern of their spouse's name over time! 
    '''),
    
    html.Div([

        html.Div([
            dcc.Dropdown(
                df['name'].unique(),
                'John',
                id='input_name', 
                placeholder="Type/Choose a first name"
            ),
            dcc.RadioItems(
                ['Count', 'Percentage'],
                'Count',
                id='crossfilter-yaxis-type',
                labelStyle={'display': 'inline-block', 'marginTop': '5px'}
            )
        ], style={'width': '48%', 'display': 'inline-block'}),
    ]),

    dcc.Graph(id='graphic')
])


@app.callback(
    Output('overall', 'figure'),
    Input('input_name', 'value'))
def update_graph(column_name):
    fig = px.scatter(top1pair_perYear, x="Marriage Year", y="Count", hover_data=['Name Pair'],color="Person A name",
                  text = 'Name Pair')
    fig.update_layout(title='Top 1 marriage name pair per year',
                  xaxis_title="Marriage Year",
    yaxis_title="Count",
    legend_title="Popular Names", height=800,width=1000)
    fig.update_traces(textposition='top center',textfont_size=9)
    return fig


@app.callback(
    Output('graphic', 'figure'),
    Input('input_name', 'value'),
Input('crossfilter-yaxis-type', 'value'))
def update_graph2(column_name, yaxis):
    dff = df[df['name'] == column_name]
    fig = px.line(dff, x="Marriage Year", y= yaxis, color="Spouse Name")
    fig.update_layout(title= column_name + "'s spouse name pattern over time")
    fig.update_yaxes(title=yaxis)
    return fig


In [171]:
app.run_server(mode="inline")